<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#朴素贝叶斯" data-toc-modified-id="朴素贝叶斯-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>朴素贝叶斯</a></span><ul class="toc-item"><li><span><a href="#数据预处理" data-toc-modified-id="数据预处理-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>数据预处理</a></span><ul class="toc-item"><li><span><a href="#做一些处理-去重-去除停用词-合并同义词-去除空白值-去除停用词也可以在分词这一步做" data-toc-modified-id="做一些处理-去重-去除停用词-合并同义词-去除空白值-去除停用词也可以在分词这一步做-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>做一些处理 去重 去除停用词 合并同义词 去除空白值 去除停用词也可以在分词这一步做</a></span></li></ul></li><li><span><a href="#jieba分词" data-toc-modified-id="jieba分词-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>jieba分词</a></span></li></ul></li><li><span><a href="#提取特征" data-toc-modified-id="提取特征-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>提取特征</a></span></li><li><span><a href="#划分数据集" data-toc-modified-id="划分数据集-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>划分数据集</a></span></li><li><span><a href="#训练模型" data-toc-modified-id="训练模型-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>训练模型</a></span></li><li><span><a href="#训练模型" data-toc-modified-id="训练模型-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>训练模型</a></span></li><li><span><a href="#测试模型" data-toc-modified-id="测试模型-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>测试模型</a></span></li><li><span><a href="#保存-分析数据" data-toc-modified-id="保存-分析数据-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>保存 分析数据</a></span></li></ul></div>

In [ ]:
# 标注数据 一定要统一标准 LDE tf-idf

In [2]:
import numpy as np
import pandas as pd

In [3]:
data = pd.read_excel(r"../data/TrainData.xlsx")
data.head()

,Comment,Sentiment
0,陈姥姥和何老师讲的特别棒，收获颇丰,1
1,很好很贴心！,1
2,有点敷衍了，内容少，连贯性一般，差强人意,0
3,视频讲解非常详细,1
4,没给代码，课件也没有,0


### 朴素贝叶斯

#### 数据预处理

##### 做一些处理 去重 去除停用词 合并同义词 去除空白值 去除停用词也可以在分词这一步做

#### jieba分词

In [4]:
import jieba
import jieba.posseg as psg
import re

dict_file = r"../stop_dic/user_dict.txt"
stop_file = r"../stop_dic/hit_stopwords.txt"

def chinese_word_cut(my_text):
    jieba.load_userdict(dict_file)
    jieba.initialize()
    
    data_cut = jieba.lcut(my_text, cut_all=False)
    # 创建一个stop_word列表，读取停用词文档，将停用词添加到列表中
    stop_words_list = []
    with open(r'../stop_dic/hit_stopwords.txt', 'r', encoding='utf-8') as fp:
        for line in fp:
            if len(line) > 0:
                stop_words_list.append(line.strip())  # 将字符串前后的空格去掉
                
    # .新建一个列表，用于存放将分析文档库与停用词库对比后去除停用词的字符串
    data_result = []
    for i in data_cut:
        if i not in stop_words_list:
            i = i.replace(' ', '')
            i = i.replace('\n', '')
            if i != '':
                data_result.append(i)
    text = ' '.join(data_result).replace('\n', ' ')
#     return " ".join(jieba.cut(my_text))
    return text
data['cut_comment'] = data.Comment.apply(chinese_word_cut)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\20866\AppData\Local\Temp\jieba.cache
Loading model cost 0.542 seconds.
Prefix dict has been built successfully.


In [5]:
data.head()

,Comment,Sentiment,cut_comment
0,陈姥姥和何老师讲的特别棒，收获颇丰,1,陈 姥姥 老师 讲 特别 棒 收获 颇丰
1,很好很贴心！,1,很 好 很 贴心
2,有点敷衍了，内容少，连贯性一般，差强人意,0,有点 敷衍 内容 少 连贯性 差强人意
3,视频讲解非常详细,1,视频 讲解 非常 详细
4,没给代码，课件也没有,0,没 代码 课件 没有


### 提取特征

In [6]:
from sklearn.feature_extraction.text import CountVectorizer  # 单词别拼错了

def get_custom_stopwords(stop_words_file):
    with open(stop_words_file, encoding='utf-8') as fp:
        stopwords = fp.read()
    stopwords_list = stopwords.split('\n')
    custom_stopwords_list = [i for i in stopwords_list]
    return custom_stopwords_list
stop_words_file = r"../stop_dic/hit_stopwords.txt"
stopwords = get_custom_stopwords(stop_words_file)
vect = CountVectorizer(max_df=0.8,  # max_df: 可用于删除过于频繁出现的术语
                      min_df=5,  #  min_df: 用于删除不经常出现的术语。
                      token_pattern=u'(?u)\\b[^\\d\\W]\\w+\\b',
                      stop_words=stopwords)

### 划分数据集

In [7]:
X = data['cut_comment']  # X表示评论的词语的一些特征 处理之前的特征
y = data.Sentiment  

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=22)

### 训练模型

In [10]:
test = pd.DataFrame(vect.fit_transform(X_train).toarray(), columns=vect.get_feature_names_out())
test.head()

D:\SoftwareInstall\anaconda\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['exp', 'lex', '①①', '①②', '①③', '①④', '①⑤', '①⑥', '①⑦', '①⑧', '①⑨', '①ａ', '①ｂ', '①ｃ', '①ｄ', '①ｅ', '①ｆ', '①ｇ', '①ｈ', '①ｉ', '①ｏ', '②①', '②②', '②③', '②④', '②⑤', '②⑥', '②⑦', '②⑧', '②⑩', '②ａ', '②ｂ', '②ｄ', '②ｅ', '②ｆ', '②ｇ', '②ｈ', '②ｉ', '②ｊ', '③①', '③⑩', '③ａ', '③ｂ', '③ｃ', '③ｄ', '③ｅ', '③ｆ', '③ｇ', '③ｈ', '④ａ', '④ｂ', '④ｃ', '④ｄ', '④ｅ', '⑤ａ', '⑤ｂ', '⑤ｄ', '⑤ｅ', '⑤ｆ', 'ｌｉ', 'ｚｘｆｉｔｌ'] not in stop_words.
  warnings.warn(


,app,c语言,good,java,kmp,linux,mooc,nice,oj,ok,...,非常适合,预习,预期,颇丰,题目,风格,风趣,高效,魅力,鼓励
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 训练模型

In [11]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

X_train_vect = vect.fit_transform(X_train)
nb.fit(X_train_vect, y_train)

train_score = nb.score(X_train_vect, y_train)
train_score

0.9279166666666666

### 测试模型

In [12]:
X_test_vect = vect.transform(X_test)
print(nb.score(X_test_vect, y_test))

0.9058333333333334


### 保存 分析数据


In [13]:
data = pd.read_excel(r"../data/TestData.xlsx")  # 里面的数据是完全没有标签的
# data.head()  # 训练集  测试集 和 待分析的数据是不一样的

In [14]:
X = data['Comment']
X_vec = vect.transform(X)

nb_result= nb.predict(X_vec)
data['nb_result'] = nb_result

In [15]:
data.head()

,Comment,nb_result
0,老师授课很详细很具体很清楚,1
1,没听明白和没理解的可以反复学习,1
2,课程内容十分充实，老师用心准备，详略得当张弛有度，一口气学一单元不觉得累，我觉得收获很大。华...,1
3,零基础学习非常棒，详细的讲解，有效的练习,1
4,好好，教的好得很,1


In [16]:
data.to_excel(r"../data/TestData.xlsx", index=False)
print('done')

done
